In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pickle

In [3]:
# pandas options
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
# try and import my own functions:
import sys
import ipynb
import re
sys.path.append('/project/Xelix_Project/utils')

## Make sure that the clean attachments etc has been run on this data

In [5]:
labelled_emails = pd.read_pickle("categorised_emails_and_threads.pkl")
print(len(labelled_emails))
bankchange_emails = labelled_emails[labelled_emails.category_slug.str.contains("bank-detail-change") == True]
len(bankchange_emails)

14333


25

In [6]:
labelled_emails.columns

Index(['id', 'subject', 'internal_date', 'from_address', 'to', 'bcc', 'cc',
       'reply_to', 'html_body', 'plain_text_body', 'attachments',
       'category_id', 'category_slug', 'has_attachment', 'html_body_as_text',
       'body'],
      dtype='object')

In [7]:
from ipynb.fs.full.Regex_html_Functions import (clean_text_re_links_brackets, find_sort_code_in_email,
                                               find_bank_vocabulary_mention_in_email, find_bank_mention_in_email,
                                               find_update_words_in_email)

In [8]:
def check_array_lengths(arrays):
    # Get the length of the first array
    the_len = len(arrays[0])
    
    # Check if all arrays have the same length
    if not all(len(l) == the_len for l in arrays):
        # Loop through the arrays
        for i in range(len(arrays)):
            # If the array is shorter than the others, print its name
            if len(arrays[i]) < the_len:
                print(f"{i}-th array has a shorter length than the others.")

In [9]:
def check_array_lengths_add_Nones(arrays):
    # Get the length of the first array
    the_len = max(len(l) for l in arrays)
    
    # Check if all arrays have the same length
    if not all(len(l) == the_len for l in arrays):
        # Loop through the arrays
        for i in range(len(arrays)):
            # If the array is shorter than the others, append 'None' until it has the same length
            while len(arrays[i]) < the_len:
                arrays[i].append(None)
    
    # Return the modified arrays list
    return arrays

In [10]:
def check_dict_value_lengths(dct):
    # Get the length of the first value
    the_len = len(next(iter(dct.values())))
    
    # Check if all values have the same length
    if not all(len(val) == the_len for val in dct.values()):
        # Loop through the values
        for key, val in dct.items():
            # If the value is shorter than the others, print its name
            if len(val) < the_len:
                print(f"{key} has a shorter length than the others.")

In [11]:
def extract_text_between_start_end(start_str, end_str, text):
    '''Function to take in text, extract the text between the start and 
    end substring, and then return the text with the inbetween text removed.'''


    start_index = text.find(start_str) + len(start_str)
    end_index = text.find(end_str)
    between_text = text[start_index:end_index]
    
    new_text = re.sub(between_text,'',text)
    
    
    return between_text, new_text

In [48]:
my_dict = {'hellpo':[1,2,3],'hellnope':[1,2]}

check_dict_value_lengths(my_dict)

hellnope has a shorter length than the others.


# Actually run through it 

In [41]:
# grow lists iteratively, then create dataframe at the end
iter_count_list = []
email_id = []
subject = []
internal_date = []
from_address = []
from_name = []
sent_to = []
sent_to_how_many = []
bcc = []
cc = []
reply_to_address = []
attachments = []
category_id = []
category_slug = []
has_attachment = []
email_text_body = []

# new columns for splitting the emails
is_part_of_thread = []
num_in_thread = []
thread_id = []

# sort code related
has_sort_code = []
sort_code = []

# language related
mentions_bank = []
mentions_change = []
mentions_account_specifics = []

# log error for normal emails too
has_error = []

# error emails:
error_mail_id = []
error_mail_body = []
error_type = []

############################
# splitting emails by this: # Disclaimer: need to check this is the same across languages etc
split_string = "From: "

iter_count = 0
error_count = 0

# change name of df here
for line in labelled_emails.itertuples():
    
    iter_count += 1
#     print(f'iter count {iter_count}, id: {line.id}')
    
    ################### if single email, we just append as we would
    if split_string not in line.body:
        
        # save thread details
        iter_count_list.append(iter_count)
        is_part_of_thread.append(False)
        num_in_thread.append(None)
        thread_id.append(None)
        
        # save data
        email_id.append(line.id)
        attachments.append(line.attachments)
        category_id.append(line.category_id)
        category_slug.append(line.category_slug)
        has_attachment.append(line.has_attachment)
        
        # email specific
        subject.append(line.subject)
        internal_date.append(line.internal_date)
        from_address.append(line.from_address)
        from_name.append(line.from_address) # just save sender email instead here
        sent_to.append(line.to)
        sent_to_how_many.append(line.to.count(',')+1)
        bcc.append(line.bcc)
        cc.append(line.cc)
        reply_to_address.append(line.reply_to)
        
        sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
        has_sort_code.append(sort_code_bool)
        sort_code.append(sort_code_num)

        # language related
        mentions_bank.append(find_bank_mention_in_email(line.body))
        mentions_change.append(find_update_words_in_email(line.body))
        mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
        
        # clean email body
        email_text_body.append(clean_text_re_links_brackets(line.body))
        has_error.append(False)
        
    ################### thread of emails
    else:
        split_emails = line.body.split("From: ")
        
        # try / except handling here
        try:
            for i, row in enumerate(split_emails):

                iter_count_list.append(iter_count)
                row = row.replace('\n',' ') # replace this so we can search for strings
                
                 # save thread details
                is_part_of_thread.append(True)
                num_in_thread.append(int(len(split_emails)-(i+1)))
                thread_id.append(str(line.id)+'_t')

                # save regardless of thread or not
                email_id.append(line.id)
                attachments.append(line.attachments)
                category_id.append(line.category_id)
                category_slug.append(line.category_slug)
                has_attachment.append(line.has_attachment)

                bcc.append(None)
                cc.append(None)
                reply_to_address.append(line.reply_to) # just keep this consistent?

                # for every split email which wasn't the first
                if i > 0:

                    # log who sent this email in thread
                    before_sent = row.split("Sent: ")[0]
                    this_email = re.findall(r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])', before_sent)
                    row = re.sub(before_sent,"",row)

                    if not(this_email):
                        this_person = before_sent
                        from_address.append(this_person) # save output
                    else:
                        this_person = re.sub(r'<(.*?)>',"",before_sent) 
                        from_address.append(this_email) # save output
                        
                    from_name.append(this_person) # save output    
                    
                    # log when this email in thread was sent
                    date_text, row = extract_text_between_start_end('Sent: ','To: ', row)
                    internal_date.append(date_text) # save output

                    # log who this email was sent to
                    to_text, row = extract_text_between_start_end('To: ','Subject: ', row)
                    to_emails = re.findall(r'<(.*?)>', to_text)
                    sent_to.append(to_emails) # save output
                    sent_to_how_many.append(len(to_emails)) # save output
            
                    # log the subject of this email
                    this_subject, row = extract_text_between_start_end('Subject: ','      ', row)
                    subject.append(this_subject) # save output
            
                    # additional cleaning if needed
                    row = re.sub('Sent: ',"",row)
                    row = re.sub('To: ',"",row)
                    row = re.sub('Subject: ',"",row)
                    row = re.sub(this_subject,"",row)

                    # conditionals
                    sort_code_bool, sort_code_num = find_sort_code_in_email(row)
                    has_sort_code.append(sort_code_bool)
                    sort_code.append(sort_code_num)

                    # language related
                    mentions_bank.append(find_bank_mention_in_email(row))
                    mentions_change.append(find_update_words_in_email(row))
                    mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

                    # clean email body
                    email_text_body.append(clean_text_re_links_brackets(row))
                    has_error.append(False) # ran without error
                
                elif i == 0:
                    
                    # first email in thread, save as if saving from the main data
                    subject.append(line.subject)
                    internal_date.append(line.internal_date)
                    from_address.append(line.from_address)
                    from_name.append(line.from_address) # just save sender email instead here
                    sent_to.append(line.to)
                    sent_to_how_many.append(line.to.count(',')+1)

                    # conditionals
                    sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
                    has_sort_code.append(sort_code_bool)
                    sort_code.append(sort_code_num)

                    # language related
                    mentions_bank.append(find_bank_mention_in_email(line.body))
                    mentions_change.append(find_update_words_in_email(line.body))
                    mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

                    # clean email body
                    email_text_body.append(clean_text_re_links_brackets(row))
                    has_error.append(False) # ran without error
                    
        # email threw an error
        except re.error as e:
            
            error_count += 1
            has_error.append(True) # ran with error
            
            # need to count if all lists are same length - error could have happened somewhere
            arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
                     bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
                     email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
                     mentions_bank,mentions_change,mentions_account_specifics,has_error]
            
            # add None's to emails with an error - this could probably be smarter
            [iter_count_list,email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
             bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
             email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
             mentions_bank,mentions_change,mentions_account_specifics,has_error] = check_array_lengths_add_Nones(arrays)
            
            # log details for error emails
            error_mail_id.append(line.id)
            error_mail_body.append(line.body) 
            error_type.append(e)
            
#             ################## if wanted ####################
#             # print out the error emails only
#             print("#" * 70, "\n\n")
#             print(f'Regex error occurred: {e}\n\n')               
#             print(f'email id: {line.id}, iter number: {iter_count}. So far there have been {error_count} errors.\n\n')
#             print(f'subject line:\t{line.subject}')
#             print(f'received from:\t{line.from_address}')
#             print(f'sent to:\t{line.to}')
#             #print(f'emails sent to: {line.to.count(',')+1} people')
#             print(f'date sent:\t{line.internal_date}')
#             print(f'slug:\t\t{line.category_slug}')
#             print("")
#             print(line.body)
#             print("\n")
#             ##################################################

print("-" * 50, "\n")
print(f'there were {error_count} errors in total.\n')
print("-" * 50)    

###################################################################### 


Regex error occurred: bad character range o-g at position 40


email id: 195490, iter number: 27. So far there have been 1 errors.


subject line:	Vow
received from:	staff@indtemps.co.uk
sent to:	{purchase.ledger@evo-group.co.uk}
date sent:	2021-10-06 08:36:46+00
slug:		invoices

Alert: There is currently a serious threat from opening email attachments. If you have any concerns about an attached file please contact IT.

Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.
Good morning,

Please find attached Invoice for w/e 19.09.2021 and w/e 26.09.2021

Kind Regards

Aiga Gangnusa

Tel: 02476 651 651

[image001 (003)]
[cid:image001.png@01D63800.D66CD360]  02476 651 651
[cid:image002.png@01D63800.D66CD360]122 Walsgrave Road, Coventry, CV2 4AX

[cid:image003.png@01D63800.D66CD360][cid:image004.png@01D63800.D66CD360][Covenant banner][cid:i

###################################################################### 


Regex error occurred: nothing to repeat at position 125


email id: 285025, iter number: 1134. So far there have been 7 errors.


subject line:	Invoices
received from:	RichardBirks@Prohiregroup.co.uk
sent to:	{Lisa.Dyson@evo-group.co.uk,Purchase.ledger@evo-group.co.uk}
date sent:	2022-05-31 12:53:45+00
slug:		invoices-subject-line-pdf

Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.

Kind Regards
Richard Birks
Senior Credit Controller




Tel:           08708 501200
Fax:       08708 501201
Calls to these numbers cost 7p per minute plus your phone company's access charge
Email:       richardbirks@prohiregroup.co.uk
Website: www.prohiregroup.co.uk

Prohire Limited | React House | Spedding Road | Fenton Industrial Estate |
Stoke-On-Trent | Staffordshire | ST4 2ST |




Any opinions expressed in this email are those of the individual a

###################################################################### 


Regex error occurred: nothing to repeat at position 118


email id: 286285, iter number: 3145. So far there have been 21 errors.


subject line:	FW: Missing invoices
received from:	Alison.Ogden@evo-group.co.uk
sent to:	{Purchase.Ledger@evo-group.co.uk}
date sent:	2022-06-06 08:21:58+00
slug:		invoices-subject-line-pdf



-----Original Message-----
From: Richard Birks <RichardBirks@Prohiregroup.co.uk> 
Sent: 01 June 2022 16:33
To: Alison Ogden <Alison.Ogden@evo-group.co.uk>
Cc: Jessica Waring <Jessica.Waring@evo-group.co.uk>; Lisa Dyson <Lisa.Dyson@evo-group.co.uk>
Subject: RE: Missing invoices

Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.

Hi Alison,

Please find enclosed all the requested copies and an updated statement,

Kind Regards
Richard Birks
Senior Credit Controller




Tel:           08708 501200
Fax:       08708 501201

###################################################################### 


Regex error occurred: nothing to repeat at position 121


email id: 286791, iter number: 3967. So far there have been 25 errors.


subject line:	Invoices
received from:	RichardBirks@Prohiregroup.co.uk
sent to:	{Lisa.Dyson@evo-group.co.uk,Purchase.ledger@evo-group.co.uk}
date sent:	2022-06-07 10:15:50+00
slug:		invoices-subject-line-pdf

Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.

Kind Regards
Richard Birks
Senior Credit Controller




Tel:           08708 501200
Fax:       08708 501201
Calls to these numbers cost 7p per minute plus your phone company's access charge
Email:       richardbirks@prohiregroup.co.uk
Website: www.prohiregroup.co.uk

Prohire Limited | React House | Spedding Road | Fenton Industrial Estate |
Stoke-On-Trent | Staffordshire | ST4 2ST |




Any opinions expressed in this email are those of the individual 

###################################################################### 


Regex error occurred: missing ), unterminated subpattern at position 484


email id: 287344, iter number: 4843. So far there have been 43 errors.


subject line:	RE: Bradford Leisure Exchange Car Park Passes
received from:	permits@horizonparking.co.uk
sent to:	{Jessica.Waring@evo-group.co.uk,purchase.ledger@evo-group.co.uk}
date sent:	2022-06-08 11:12:13+00
slug:		reminder-content

Attention: This email originated outside of our organisation. Please be extra vigilant when opening attachments or clicking links.

Good afternoon



Please find attached your June 22 invoice



Outstanding payment for invoice P4269


Kind Regards

Darren Ambrose


[Horizon Parking Limited]<http://www.horizonparking.co.uk>
Permits Department​
Horizon Parking Limited
P:      01245 392 289<tel:01245%20392%20289>
E:      permits@horizonparking.co.uk<mailto:permits@horizonparking.co.uk>        |      W:      www.horizonparking.co.uk<http:/

...

###################################################################### 


Regex error occurred: bad character range E-2 at position 52


email id: 365963, iter number: 12592. So far there have been 96 errors.


subject line:	RE: Unapplied cash on account [Account: 326292009809] [OE-2550126]
received from:	Mike.Daugherty@scripps.com
sent to:	{aws-receivables-support+W0FjY291bnQ6MzI2MjkyMDA5ODA5XVtPRS0yNTUwMTI2XQ==@email.amazon.com,APCustomerService@scripps.com}
date sent:	2021-08-19 17:04:26+00
slug:		no-action

Hello.  Yes, that is acceptable.  Thanks.  Mike  Mike Daugherty A/P Supervisor (513) 977-5114  "Give light and the people will find their own way"     From: Randall Alberto Hernandez Chacon <aws-receivables-support@amazon.com> Sent: Thursday, August 19, 2021 2:29 AM To: Appids_Corp_AP Customer Service <APCustomerService@scripps.com> Cc: Daugherty, Mike <Mike.Daugherty@scripps.com> Subject: Unapplied cash on account [Account: 326292009809] [OE-2550126]  [EXTERNAL SENDER]  Hello M

###################################################################### 


Regex error occurred: unbalanced parenthesis at position 716


email id: 373618, iter number: 12744. So far there have been 101 errors.


subject line:	RE: Sale of ION Media to The E.W. Scripps Company effective January 7, 2021
received from:	ndye@ddrealty.net
sent to:	{APCustomerService@scripps.com}
date sent:	2021-04-08 17:33:16+00
slug:		statement

[EXTERNAL SENDER]  See attached.   Nicholas Dye D&D Realty Group 11 W Market Street 3rd Floor Wilkes-Barre, PA 18701 Office:  570-704-2343 ext 101 Direct:  215-582-2834 Email: ndye@ddrealty.net  From: Appids_Corp_AP Customer Service <APCustomerService@scripps.com> Sent: Thursday, April 8, 2021 1:25 PM To: Nicholas Dye <ndye@ddrealty.net> Subject: FW: Sale of ION Media to The E.W. Scripps Company effective January 7, 2021  Please see the comments from our Procurement team. Please send us a copy as soon as possible. Thank you, Moses  From: Bell, Tracey < tbbell@scripp

###################################################################### 


Regex error occurred: bad character range E-4 at position 96


email id: 381933, iter number: 12912. So far there have been 106 errors.


subject line:	RE: RE: AWS Marketplace Statement Available [Account: 326292009809] [Statement ID: 571630137] [IE-4005827]
received from:	Mike.Daugherty@scripps.com
sent to:	{aws-receivables-support+W0FjY291bnQ6MzI2MjkyMDA5ODA5XVtJRS00MDA1ODI3XQ==@email.amazon.com,APCustomerService@scripps.com}
date sent:	2020-10-08 12:11:56+00
slug:		other-queries

350176089 paid via EFT on  12/27/19 365644941 paid via EFT on 01/27/20  Also, please note that normalinvoices@scripps.com is an unmanned email box for the processing of invoices.  Please only send only invoices for payment to that box.  All request for information need to be sent to apcustomerservice@scripps.com   Thanks.  Mike  Mike Daugherty A/P Supervisor (513) 977-5114  "Give light and the people will find their own way"     From:

###################################################################### 


Regex error occurred: multiple repeat at position 458


email id: 383615, iter number: 13040. So far there have been 111 errors.


subject line:	Re: Untitled ticket 334587- REFUND DUE
received from:	christoph@blinkist.com
sent to:	{Jacque.McFalls@scripps.com}
date sent:	2020-09-01 13:48:08+00
slug:		no-action

[EXTERNAL SENDER]  Please find attached the filled out and signed form. Looking forward to your confirmation of receipt. Thanks, Christoph -- Kind regards / Freundliche Grüße, Christoph Zeller Finance & Legal Affairs  Sonnenallee 223 | 12059 Berlin | Germany [maps.google.com]  Blinkist [blinkist.com] - Big ideas in small packages On Tue, Sep 1, 2020 at 3:32 PM McFalls, Jacque < Jacque.McFalls@scripps.com > wrote: Thank you for your quick response.  We will look out for the completed form later today.  Regards,  Jacquelyn McFalls Supervisor Order to Cash Accounting Operations The E.W Scripps Company 312 Waln

-------------------------------------------------- 

there were 122 errors in total.

--------------------------------------------------


In [42]:
arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
                     bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
                     email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
                     mentions_bank,mentions_change,mentions_account_specifics,has_error]
            
check_array_lengths(arrays)

In [43]:
dict_data = {'iter_count_list':iter_count_list,'email_id':email_id,'is_part_of_thread':is_part_of_thread,'num_in_thread':num_in_thread,'thread_id':thread_id,
             'has_error':has_error,
             'internal_date':internal_date,'category_id':category_id,'category_slug':category_slug,'subject':subject,
            'has_attachment':has_attachment,'has_sort_code':has_sort_code,'sort_code':sort_code,
             'mentions_bank':mentions_bank,'mentions_change':mentions_change,'mentions_account_specifics':mentions_account_specifics,
             'from_address':from_address,'from_name':from_name,'sent_to':sent_to,'sent_to_how_many':sent_to_how_many,
             'bcc':bcc,'cc':cc,'reply_to_address':reply_to_address,'attachments':attachments,'email_text_body':email_text_body
            }

In [44]:
new_df = pd.DataFrame.from_dict(dict_data)

In [45]:
new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']] = new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']].fillna(0).astype(int)

In [46]:
new_df.to_csv('unwrapped_labelled_emails.csv')

In [47]:
new_df

,iter_count_list,email_id,is_part_of_thread,num_in_thread,thread_id,has_error,internal_date,category_id,category_slug,subject,has_attachment,has_sort_code,sort_code,mentions_bank,mentions_change,mentions_account_specifics,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,attachments,email_text_body
0,1,167529,False,0,None,False,2021-11-17 16:08:00+00,100,invoices,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,False,False,None,False,False,False,sales@bluestemgroup.co.uk,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},None,Attention: This email originated outside of...
1,2,167544,False,0,None,False,2022-07-05 15:37:48+00,1,statement,"Overdue Account £59,632.58",True,False,None,False,False,False,sefika@albion.co.uk,sefika@albion.co.uk,{Matt.Hales@evo-group.co.uk},1,{},{Ryan.Leigh@evo-group.co.uk},{},Followups(6).pdf,Attention: This email originated outside o...
2,3,170741,True,9,170741_t,False,2021-07-28 10:10:32+00,100,invoices,FW: APRIL ACCOUNT,False,False,None,False,True,False,Lisa.Dyson@evo-group.co.uk,Lisa.Dyson@evo-group.co.uk,{Purchase.Ledger@evo-group.co.uk},1,None,None,{},None,Lisa Dyson Group Accounts Payable Su...
3,3,170741,True,8,170741_t,False,28 July 2021 10:53,100,invoices,RE: APRIL ACCOUNT,False,False,None,False,False,False,[peter.bown@slingsby.com],Peter Bown,[Lisa.Dyson@evo-group.co.uk],1,None,None,{},None,Attention: This email originated outside...
4,3,170741,True,7,170741_t,False,28 July 2021 10:35,100,invoices,RE: APRIL ACCOUNT,False,False,None,False,False,False,[ Lisa.Dyson@evo-group.co.uk ],Lisa Dyson,[ peter.bown@slingsby.com ],1,None,None,{},None,Thanks Peter Could you send me the c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17818,14333,406564,True,4,406564_t,False,"Friday, December 17, 2021 1:48 AM",133,demo,Statement of Account for your Amazon Advertisi...,False,False,None,False,False,False,"Rudramayee, Soumya","Rudramayee, Soumya","[ tratliff@scrippsnetworks.com , APCustomerse...",2,None,None,{},None,Statement of Account for your Amazon Advertisi...
17819,14333,406564,True,3,406564_t,False,"Friday, December 10, 2021 2:09 AM",133,demo,Statement of Account for your Amazon Advertisi...,False,False,None,False,False,False,"Rudramayee, Soumya","Rudramayee, Soumya",[],0,None,None,{},None,Statement of Account for your Amazon Advertisi...
17820,14333,406564,True,2,406564_t,False,"Saturday, November 27, 2021 12:21 AM",133,demo,Statement of Account for your Amazon Advertisi...,False,False,None,False,False,False,"Rudramayee, Soumya","Rudramayee, Soumya",[],0,None,None,{},None,Statement of Account for your Amazon Advertisi...
17821,14333,406564,True,1,406564_t,False,"Saturday, November 20, 2021 1:51 AM",133,demo,Statement of Account for your Amazon Advertisi...,False,False,None,False,False,False,"Rudramayee, Soumya","Rudramayee, Soumya",[],0,None,None,{},None,Statement of Account for your Amazon Advertisi...


# Examples of emails not easily processed

In [ ]:
example = labelled_emails[labelled_emails['id'] == 211263]
example['body'].iloc[0]

In [ ]:
example = labelled_emails[labelled_emails['id'] == 195490]
example['body'].iloc[0]

In [ ]:
text = example['body'].iloc[0].replace('\n',' ').replace('/',' ')
text

## Looking at emails some more

In [ ]:
bankchange_emails["attachments"].unique()

In [ ]:
from ipynb.fs.full.Regex_html_Functions import (clean_text_re_links_brackets, find_sort_code_in_email,
                                               find_bank_vocabulary_mention_in_email, find_bank_mention_in_email,
                                               find_update_words_in_email)

In [ ]:
# look at the emails
# iter_count = 1
# for row in labelled_emails[0:100].itertuples():
#     print("#" * 70, "\n\n")
#     print(f'email num: {iter_count}, email id: {row.id}')
#     iter_count += 1
#     print(f'subject line:\t{row.subject}')
#     print(f'received from:\t{row.from_address}')
#     print(f'sent to:\t{row.to}')
#     print(f'date sent:\t{row.internal_date}')
#     print(f'slug:\t\t{row.category_slug}')
#     print(row.body)

# Look at the emails in split form

# insert try/except error handling

In [ ]:
# def check_array_lengths(arrays):
#     # Get the length of the first array
#     the_len = len(arrays[0])
    
#     # Check if all arrays have the same length
#     if not all(len(l) == the_len for l in arrays):
#         # Loop through the arrays
#         for i in range(len(arrays)):
#             # If the array is shorter than the others, print its name
#             if len(arrays[i]) < the_len:
#                 print(f"{i}-th array has a shorter length than the others.")
    


In [ ]:
# def check_dict_value_lengths(dct):
#     # Get the length of the first value
#     the_len = len(next(iter(dct.values())))
    
#     # Check if all values have the same length
#     if not all(len(val) == the_len for val in dct.values()):
#         # Loop through the values
#         for key, val in dct.items():
#             # If the value is shorter than the others, print its name
#             if len(val) < the_len:
#                 print(f"{key} has a shorter length than the others.")

In [ ]:
# def check_array_lengths_add_Nones(arrays):
#     # Get the length of the first array
#     the_len = max(len(l) for l in arrays)
    
#     # Check if all arrays have the same length
#     if not all(len(l) == the_len for l in arrays):
#         # Loop through the arrays
#         for i in range(len(arrays)):
#             # If the array is shorter than the others, append 'None' until it has the same length
#             while len(arrays[i]) < the_len:
#                 arrays[i].append(None)
    
#     # Return the modified arrays list
#     return arrays

Actually, try to iterate through it first, without saving anything, just so I can chuck out the error ones?

## Use below to check

In [ ]:
# # grow lists iteratively, then create dataframe at the end
# iter_count_list = []
# email_id = []
# subject = []
# internal_date = []
# from_address = []
# from_name = []
# sent_to = []
# sent_to_how_many = []
# bcc = []
# cc = []
# reply_to_address = []
# attachments = []
# category_id = []
# category_slug = []
# has_attachment = []
# email_text_body = []

# # new columns for splitting the emails
# is_part_of_thread = []
# num_in_thread = []
# thread_id = []

# # sort code related
# has_sort_code = []
# sort_code = []

# # language related
# mentions_bank = []
# mentions_change = []
# mentions_account_specifics = []

# # log error for normal emails too
# has_error = []

# # error emails:
# error_mail_id = []
# error_mail_body = []

# ############################
# # splitting emails by this: # Disclaimer: need to check this is the same across languages etc
# split_string = "From: "

# iter_count = 1
# error_count = 0

# # change name of df here
# for line in labelled_emails[0:100].itertuples():
    
#     iter_count += 1
    
#     ################### if single email, we just append as we would
#     if split_string not in line.body:
        
#         # save thread details
#         iter_count_list.append(iter_count)
#         is_part_of_thread.append(0)
#         num_in_thread.append(None)
#         thread_id.append(None)
        
#         # save data
#         email_id.append(line.id)
#         attachments.append(line.attachments)
#         category_id.append(line.category_id)
#         category_slug.append(line.category_slug)
#         has_attachment.append(line.has_attachment)
        
#         # email specific
#         subject.append(line.subject)
#         internal_date.append(line.internal_date)
#         from_address.append(line.from_address)
#         from_name.append(line.from_address) # just save sender email instead here
#         sent_to.append(line.to)
#         sent_to_how_many.append(line.to.count(',')+1)
#         bcc.append(line.bcc)
#         cc.append(line.cc)
#         reply_to_address.append(line.reply_to)
        
#         sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
#         has_sort_code.append(sort_code_bool)
#         sort_code.append(sort_code_num)

#         # language related
#         mentions_bank.append(find_bank_mention_in_email(line.body))
#         mentions_change.append(find_update_words_in_email(line.body))
#         mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
        
#         # clean email body
#         email_text_body.append(clean_text_re_links_brackets(line.body))
#         has_error.append(0)
        
#     ################### thread of emails
#     else:
#         split_emails = line.body.split("From: ")
        
#         # try / except handling here
#         try:
#             for i, row in enumerate(split_emails):

#                 iter_count_list.append(iter_count)
#                 row = row.replace('/',' ')
#                  # save thread details
#                 is_part_of_thread.append(1)
#                 num_in_thread.append(int(len(split_emails)-(i+1)))
#                 thread_id.append(str(line.id)+'_t')

#                 # save regardless of thread or not
#                 email_id.append(line.id)
#                 attachments.append(line.attachments)
#                 category_id.append(line.category_id)
#                 category_slug.append(line.category_slug)
#                 has_attachment.append(line.has_attachment)

#                 bcc.append(None)
#                 cc.append(None)
#                 reply_to_address.append(line.reply_to) # just keep this consistent?

#                 # for every split email which wasn't the first
#                 if i > 0:

#                     before_sent = row.split("Sent: ")[0]
# #                     print(f'before sent: {before_sent}\n')
#                     #print(f'From: {this_person}, {this_email}')
#                     this_email = re.findall(r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])', before_sent)
# #                     print(f'from email:\t{this_email}')

# #                     print(f'body: {row}')
#                     row = re.sub(before_sent,"",row)
#                     #row = row.split("Sent: ")[1]

#                     if not(this_email):
#                         this_person = before_sent
#                         from_address.append(this_person) # save output
#                     else:
#                         from_address.append(this_email) # save output
#                         this_person = re.sub(r'<(.*?)>',"",before_sent)  

# #                     print(f'from person:\t{this_person}')
#                     from_name.append(this_person) # save output
#                     start_index = row.find('Sent: ') + len('Sent: ')
#                     end_index = row.find('To: ')
#                     date_text = row[start_index:end_index]
#                     internal_date.append(date_text) # save output

#                     row = re.sub(date_text,"",row)
#                     #row = row.split('To: ')[1]
# #                     print(f'Date:\t\t{date_text}')

#                     start_index = row.find('To: ') + len('To: ')
#                     end_index = row.find('Subject: ')
#                     to_text = row[start_index:end_index]
#                     to_emails = re.findall(r'<(.*?)>', to_text)
#                     sent_to.append(to_emails) # save output
#                     row = re.sub(to_text,"",row)

# #                     print(f'To:\t\t{to_emails}')
# #                     print(f'emails sent to: {len(to_emails)} people')
#                     sent_to_how_many.append(len(to_emails)) # save output
#                     start_index = row.find('Subject: ') + len('Subject: ')
#                     end_index = row.find('      ')
#                     this_subject = row[start_index:end_index]
# #                     print(f'subject:\t{this_subject}')
#                     subject.append(this_subject) # save output


#                     row = re.sub('Sent: ',"",row)
#                     row = re.sub('To: ',"",row)
#                     row = re.sub('Subject: ',"",row)
#                     row = re.sub(this_subject,"",row)
                    
# #                     # save the text details
# #                     from_name.append(this_person) # save output

# #                     if not(this_email):
# #                         this_person = before_sent
# #                         from_address.append(this_person) # save output
# #                     else:
# #                         from_address.append(this_email) # save output
# #                         this_person = re.sub(r'<(.*?)>',"",before_sent)  
                        
# #                     internal_date.append(date_text) # save output
# #                     sent_to.append(to_emails) # save output
# #                     sent_to_how_many.append(len(to_emails)) # save output
# #                     subject.append(this_subject) # save output

#                     # conditionals
#                     sort_code_bool, sort_code_num = find_sort_code_in_email(row)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(row))
#                     mentions_change.append(find_update_words_in_email(row))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(row))
        
#                     has_error.append(0) # ran without error
                
#                 elif i == 0:
                    
#                     # first email in thread, save as if saving from the main data
#                     subject.append(line.subject)
#                     internal_date.append(line.internal_date)
#                     from_address.append(line.from_address)
#                     from_name.append(line.from_address) # just save sender email instead here
#                     sent_to.append(line.to)
#                     sent_to_how_many.append(line.to.count(',')+1)

#                     sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(line.body))
#                     mentions_change.append(find_update_words_in_email(line.body))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(row))
                    
#                     has_error.append(0) # ran without error

                    
#         # email threw an error
#         except Exception as e:
#             error_count += 1
#             has_error.append(1) # ran with error
            
#             # need to count if all lists are same length - error could have happened somewhere
#             arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#                      bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#                      email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#                      mentions_bank,mentions_change,mentions_account_specifics,has_error]
            
#             # add None's to emails with an error
#             [iter_count_list,email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#              bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#              email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#              mentions_bank,mentions_change,mentions_account_specifics,has_error] = check_array_lengths_add_Nones(arrays)
            
#             # print out the error emails only
#             print("#" * 70, "\n\n")
#             print(f'\n\n\n\nemail with id: {line.id}, iter number: {iter_count}, threw an error. So far there have been {error_count} errors.\n\n\n\n')
#             error_mail_id.append(line.id)
#             error_mail_body.append(line.body) 
            
#             print(f'subject line:\t{line.subject}')
#             print(f'received from:\t{line.from_address}')
#             print(f'sent to:\t{line.to}')
#             #print(f'emails sent to: {line.to.count(',')+1} people')
#             print(f'date sent:\t{line.internal_date}')
#             print(f'slug:\t\t{line.category_slug}')
#             print("")
#             print(line.body)
#             print("\n")

# print(f'there were {error_count} errors in total.')

## Latest version of this script so far

In [ ]:
# # grow lists iteratively, then create dataframe at the end
# iter_count_list = []
# email_id = []
# subject = []
# internal_date = []
# from_address = []
# from_name = []
# sent_to = []
# sent_to_how_many = []
# bcc = []
# cc = []
# reply_to_address = []
# attachments = []
# category_id = []
# category_slug = []
# has_attachment = []
# email_text_body = []

# # new columns for splitting the emails
# is_part_of_thread = []
# num_in_thread = []
# thread_id = []

# # sort code related
# has_sort_code = []
# sort_code = []

# # language related
# mentions_bank = []
# mentions_change = []
# mentions_account_specifics = []

# # log error for normal emails too
# has_error = []

# # error emails:
# error_mail_id = []
# error_mail_body = []
# error_type = []

# ############################
# # splitting emails by this: # Disclaimer: need to check this is the same across languages etc
# split_string = "From: "

# iter_count = 0
# error_count = 0

# # change name of df here
# for line in labelled_emails[0:200].itertuples():
    
#     iter_count += 1
    
#     ################### if single email, we just append as we would
#     if split_string not in line.body:
        
#         # save thread details
#         iter_count_list.append(iter_count)
#         is_part_of_thread.append(0)
#         num_in_thread.append(None)
#         thread_id.append(None)
        
#         # save data
#         email_id.append(line.id)
#         attachments.append(line.attachments)
#         category_id.append(line.category_id)
#         category_slug.append(line.category_slug)
#         has_attachment.append(line.has_attachment)
        
#         # email specific
#         subject.append(line.subject)
#         internal_date.append(line.internal_date)
#         from_address.append(line.from_address)
#         from_name.append(line.from_address) # just save sender email instead here
#         sent_to.append(line.to)
#         sent_to_how_many.append(line.to.count(',')+1)
#         bcc.append(line.bcc)
#         cc.append(line.cc)
#         reply_to_address.append(line.reply_to)
        
#         sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
#         has_sort_code.append(sort_code_bool)
#         sort_code.append(sort_code_num)

#         # language related
#         mentions_bank.append(find_bank_mention_in_email(line.body))
#         mentions_change.append(find_update_words_in_email(line.body))
#         mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
        
#         # clean email body
#         email_text_body.append(clean_text_re_links_brackets(line.body))
#         has_error.append(0)
        
#     ################### thread of emails
#     else:
#         split_emails = line.body.split("From: ")
        
#         # try / except handling here
#         try:
#             for i, row in enumerate(split_emails):

#                 iter_count_list.append(iter_count)
#                 #row = row.replace('/',' ')
                
#                  # save thread details
#                 is_part_of_thread.append(1)
#                 num_in_thread.append(int(len(split_emails)-(i+1)))
#                 thread_id.append(str(line.id)+'_t')

#                 # save regardless of thread or not
#                 email_id.append(line.id)
#                 attachments.append(line.attachments)
#                 category_id.append(line.category_id)
#                 category_slug.append(line.category_slug)
#                 has_attachment.append(line.has_attachment)

#                 bcc.append(None)
#                 cc.append(None)
#                 reply_to_address.append(line.reply_to) # just keep this consistent?

#                 # for every split email which wasn't the first
#                 if i > 0:

#                     before_sent = row.split("Sent: ")[0]
# #                     print(f'before sent: {before_sent}\n')
#                     #print(f'From: {this_person}, {this_email}')
#                     this_email = re.findall(r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])', before_sent)
# #                     print(f'from email:\t{this_email}')

# #                     print(f'body: {row}')
#                     #row = re.sub(before_sent,"",row)
#                     row = row.split("Sent: ")[1]

#                     if not(this_email):
#                         this_person = before_sent
#                         from_address.append(this_person) # save output
#                     else:
#                         this_person = re.sub(r'<(.*?)>',"",before_sent) 
#                         from_address.append(this_email) # save output
                        
#                     #print(f'from person:\t{this_person}')
#                     from_name.append(this_person) # save output
                    
#                     start_index = row.find('Sent: ') + len('Sent: ')
#                     end_index = row.find('To: ')
#                     date_text = row[start_index:end_index]
#                     internal_date.append(date_text) # save output

#                     #row = re.sub(date_text,"",row)
#                     row = row.split('To: ')[1]
# #                     print(f'Date:\t\t{date_text}')

#                     start_index = row.find('To: ') + len('To: ')
#                     end_index = row.find('Subject: ')
#                     to_text = row[start_index:end_index]
#                     to_emails = re.findall(r'<(.*?)>', to_text)
#                     sent_to.append(to_emails) # save output
#                     row = re.sub(to_text,"",row)

# #                     print(f'To:\t\t{to_emails}')
# #                     print(f'emails sent to: {len(to_emails)} people')
#                     sent_to_how_many.append(len(to_emails)) # save output
#                     start_index = row.find('Subject: ') + len('Subject: ')
#                     end_index = row.find('      ')
#                     this_subject = row[start_index:end_index]
# #                     print(f'subject:\t{this_subject}')
#                     subject.append(this_subject) # save output


#                     row = re.sub('Sent: ',"",row)
#                     row = re.sub('To: ',"",row)
#                     row = re.sub('Subject: ',"",row)
#                     row = re.sub(this_subject,"",row)
                    
# #                     # save the text details
# #                     from_name.append(this_person) # save output

# #                     if not(this_email):
# #                         this_person = before_sent
# #                         from_address.append(this_person) # save output
# #                     else:
# #                         from_address.append(this_email) # save output
# #                         this_person = re.sub(r'<(.*?)>',"",before_sent)  
                        
# #                     internal_date.append(date_text) # save output
# #                     sent_to.append(to_emails) # save output
# #                     sent_to_how_many.append(len(to_emails)) # save output
# #                     subject.append(this_subject) # save output

#                     # conditionals
#                     sort_code_bool, sort_code_num = find_sort_code_in_email(row)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(row))
#                     mentions_change.append(find_update_words_in_email(row))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(row))
        
#                     has_error.append(0) # ran without error
                
#                 elif i == 0:
                    
#                     # first email in thread, save as if saving from the main data
#                     subject.append(line.subject)
#                     internal_date.append(line.internal_date)
#                     from_address.append(line.from_address)
#                     from_name.append(line.from_address) # just save sender email instead here
#                     sent_to.append(line.to)
#                     sent_to_how_many.append(line.to.count(',')+1)

#                     sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(line.body))
#                     mentions_change.append(find_update_words_in_email(line.body))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(row))
                    
#                     has_error.append(0) # ran without error

                    
#         # email threw an error
#         except re.error as e:
            

            
#             error_count += 1
#             has_error.append(1) # ran with error
            
#             # need to count if all lists are same length - error could have happened somewhere
#             arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#                      bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#                      email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#                      mentions_bank,mentions_change,mentions_account_specifics,has_error]
            
#             # add None's to emails with an error
#             [iter_count_list,email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#              bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#              email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#              mentions_bank,mentions_change,mentions_account_specifics,has_error] = check_array_lengths_add_Nones(arrays)
            
            
#             error_mail_id.append(line.id)
#             error_mail_body.append(line.body) 
#             error_type.append(e)
            
#             # print out the error emails only
#             print("#" * 70, "\n\n")
#             print(f'Regex error occurred: {e}\n\n')               
#             print(f'email id: {line.id}, iter number: {iter_count}. So far there have been {error_count} errors.\n\n')
#             print(f'subject line:\t{line.subject}')
#             print(f'received from:\t{line.from_address}')
#             print(f'sent to:\t{line.to}')
#             #print(f'emails sent to: {line.to.count(',')+1} people')
#             print(f'date sent:\t{line.internal_date}')
#             print(f'slug:\t\t{line.category_slug}')
#             print("")
#             print(line.body)
#             print("\n")

# print("-" * 50, "\n")
# print(f'there were {error_count} errors in total.\n')
# print("-" * 50)    

# Check if all arrays same length

In [ ]:
arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
                     bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
                     email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
                     mentions_bank,mentions_change,mentions_account_specifics,has_error]
#             
check_array_lengths(arrays)

# if no error, create dictionary from lists and save dataframe

In [ ]:
dict_data = {'iter_count_list':iter_count_list,'email_id':email_id,'is_part_of_thread':is_part_of_thread,'num_in_thread':num_in_thread,'thread_id':thread_id,
             'has_error':has_error,
             'internal_date':internal_date,'category_id':category_id,'category_slug':category_slug,'subject':subject,
            'has_attachment':has_attachment,'has_sort_code':has_sort_code,'sort_code':sort_code,
             'mentions_bank':mentions_bank,'mentions_change':mentions_change,'mentions_account_specifics':mentions_account_specifics,
             'from_address':from_address,'from_name':from_name,'sent_to':sent_to,'sent_to_how_many':sent_to_how_many,
             'bcc':bcc,'cc':cc,'reply_to_address':reply_to_address,'attachments':attachments,'email_text_body':email_text_body
            }

print(len(dict_data))

In [ ]:
new_df = pd.DataFrame.from_dict(dict_data)

In [ ]:
print(len(new_df))
new_df.head(261)

In [ ]:
new_df.to_pickle("labelled_emails_split_newfeatures.pkl")

In [ ]:
error_dict_data = {'email_id':error_mail_id,
                   'email_text_body':error_mail_body,'error_type':error_type,
            }

print(len(error_dict_data))

In [ ]:
error_df = pd.DataFrame.from_dict(error_dict_data)
print(len(error_df))
error_df.head()

In [ ]:
error_df.to_pickle("labelled_emails_errors.pkl")

## Some more Regex cleaning - I'm checking here

Check if:
- what is going wrong with cleaning the "before sent" part of string?


In [ ]:
# import re

# # Define a regular expression pattern that matches any of the prefixes
# # prefix_pattern = re.compile('(Subject:|RE:|FW:)\s+')
# # pattern_2 = re.compile('To: ')

# # to_emails = re.findall(r'<(.*?)>', to_text)
# #                     sent_to.append(to_emails) # save output
# #                     row = re.sub(to_text,"",row)

# def extract_text_between_start_end(start_str, end_str, text):
#     '''Function to take in text, extract the text between the start and 
#     end substring, and then return the text with the inbetween text removed.'''

#     # Search for the pattern in the row
#     match = start_str.search(text)
    
#     if match:
#         # Extract the matched prefix from the row
#         prefix = match.group(1)
        
#         # Remove the prefix and any leading whitespace from the row
#         start_index = match.end()
        
#         row = row[start_index:].lstrip()
        
#         # Find the index of the 'To:' header, if present
#         end_index = end_str.search(row)
        
#         if end_match:
#             # Extract the 'To:' text between the prefix and the 'Subject:' or end of line
#             start_index = end_match.end()
#             start_match = start_str.search(row[start_index:])
#             if start_match:
#                 end_index = start_match.start()
#             else:
#                 end_index = len(row[start_index:])
#             in_between_text = row[start_index:start_index + end_index].strip()
#         else:
#             # 'To:' header not present, use empty string
#             in_between_text = ''
        
#         new_text = re.sub(in_between_text,'',text)
        
#         # Do something with the extracted data (e.g., print it)
#         return in_between_text, new_text

In [ ]:
example = labelled_emails[labelled_emails['id'] == 195490]
a = example['body'].iloc[0]
a

In [ ]:
# def extract_text_between_start_end(start_str, end_str, text):
#     '''Function to take in text, extract the text between the start and 
#     end substring, and then return the text with the inbetween text removed.'''
    
# #     # Convert string start_str to compiled regex object if needed
# #     if isinstance(start_str, str):
# #         start_str = re.compile(start_str)
    
# #     # Convert string end_str to compiled regex object if needed
# #     if isinstance(end_str, str):
# #         end_str = re.compile(end_str)

#     # Search for the pattern in the text
#     match = start_str.search(text)
    
#     if match:
#         # Extract the matched prefix from the text
#         prefix = match.group(1)
        
#         # Remove the prefix and any leading whitespace from the text
#         start_index = match.end()
#         text = text[start_index:].lstrip()
        
#         # Find the index of the end_str, if present
#         end_index = end_str.search(text)
#         if end_index:
#             end_index = end_index.start()
#         else:
#             end_index = len(text)
        
#         # Extract the in-between text
#         in_between_text = text[:end_index].strip()
        
#         # Return the in-between text and the original text with in-between text removed
#         return in_between_text, text[:match.start()] + text[end_index:]
#     else:
#         # Return None if start_str not found in text
#         return None, text

In [19]:
def extract_text_between_start_end(start_str, end_str, text):
    '''Function to take in text, extract the text between the start and 
    end substring, and then return the text with the inbetween text removed.'''


    start_index = text.find(start_str) + len(start_str)
    end_index = text.find(end_str)
    between_text = text[start_index:end_index]
    
    text = re.sub(between_text,'',text)
    
    
    return between_text, text

In [ ]:
#example = labelled_emails[labelled_emails['id'] == 195490]
#a = labelled_emails['body'].iloc[2]
#a

In [ ]:
# #start_pattern = re.compile('Subject: |RE: |FW: ')
# #end_pattern = re.compile('To: ')

# start_pattern = 'Subject: '
# end_pattern = '   '

# a = re.sub('\n',' ', a)

# newa, newb = extract_text_between_start_end(start_pattern,end_pattern, a)

In [ ]:
# # grow lists iteratively, then create dataframe at the end
# iter_count_list = []
# email_id = []
# subject = []
# internal_date = []
# from_address = []
# from_name = []
# sent_to = []
# sent_to_how_many = []
# bcc = []
# cc = []
# reply_to_address = []
# attachments = []
# category_id = []
# category_slug = []
# has_attachment = []
# email_text_body = []

# # new columns for splitting the emails
# is_part_of_thread = []
# num_in_thread = []
# thread_id = []

# # sort code related
# has_sort_code = []
# sort_code = []

# # language related
# mentions_bank = []
# mentions_change = []
# mentions_account_specifics = []

# # log error for normal emails too
# has_error = []

# # error emails:
# error_mail_id = []
# error_mail_body = []
# error_type = []

# ############################
# # splitting emails by this: # Disclaimer: need to check this is the same across languages etc
# split_string = "From: "

# iter_count = 0
# error_count = 0

# # change name of df here
# for line in labelled_emails.itertuples():
    
#     iter_count += 1
# #     print(f'iter count {iter_count}, id: {line.id}')
    
#     ################### if single email, we just append as we would
#     if split_string not in line.body:
        
#         # save thread details
#         iter_count_list.append(iter_count)
#         is_part_of_thread.append(False)
#         num_in_thread.append(None)
#         thread_id.append(None)
        
#         # save data
#         email_id.append(line.id)
#         attachments.append(line.attachments)
#         category_id.append(line.category_id)
#         category_slug.append(line.category_slug)
#         has_attachment.append(line.has_attachment)
        
#         # email specific
#         subject.append(line.subject)
#         internal_date.append(line.internal_date)
#         from_address.append(line.from_address)
#         from_name.append(line.from_address) # just save sender email instead here
#         sent_to.append(line.to)
#         sent_to_how_many.append(line.to.count(',')+1)
#         bcc.append(line.bcc)
#         cc.append(line.cc)
#         reply_to_address.append(line.reply_to)
        
#         sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
#         has_sort_code.append(sort_code_bool)
#         sort_code.append(sort_code_num)

#         # language related
#         mentions_bank.append(find_bank_mention_in_email(line.body))
#         mentions_change.append(find_update_words_in_email(line.body))
#         mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
        
#         # clean email body
#         email_text_body.append(clean_text_re_links_brackets(line.body))
#         has_error.append(False)
        
#     ################### thread of emails
#     else:
#         split_emails = line.body.split("From: ")
        
#         # try / except handling here
#         try:
#             for i, row in enumerate(split_emails):

#                 iter_count_list.append(iter_count)
#                 row = row.replace('\n',' ') # replace this so we can search for strings
                
#                  # save thread details
#                 is_part_of_thread.append(True)
#                 num_in_thread.append(int(len(split_emails)-(i+1)))
#                 thread_id.append(str(line.id)+'_t')

#                 # save regardless of thread or not
#                 email_id.append(line.id)
#                 attachments.append(line.attachments)
#                 category_id.append(line.category_id)
#                 category_slug.append(line.category_slug)
#                 has_attachment.append(line.has_attachment)

#                 bcc.append(None)
#                 cc.append(None)
#                 reply_to_address.append(line.reply_to) # just keep this consistent?

#                 # for every split email which wasn't the first
#                 if i > 0:

#                     before_sent = row.split("Sent: ")[0]
# #                     print(f'before sent: {before_sent}\n')
#                     #print(f'From: {this_person}, {this_email}')
#                     this_email = re.findall(r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])', before_sent)
# #                     print(f'from email:\t{this_email}')

# #                     print(f'body: {row}')
#                     row = re.sub(before_sent,"",row)
#                     #row = row.split("Sent: ")[1]

#                     if not(this_email):
#                         this_person = before_sent
#                         from_address.append(this_person) # save output
#                     else:
#                         this_person = re.sub(r'<(.*?)>',"",before_sent) 
#                         from_address.append(this_email) # save output
                        
#                     #print(f'from person:\t{this_person}')
#                     from_name.append(this_person) # save output
                    
#                     start_index = row.find('Sent: ') + len('Sent: ')
#                     end_index = row.find('To: ')
#                     date_text = row[start_index:end_index]
#                     internal_date.append(date_text) # save output

#                     row = re.sub(date_text,"",row)
#                     #row = row.split('To: ')[1]
# #                     print(f'Date:\t\t{date_text}')

#                     start_index = row.find('To: ') + len('To: ')
#                     end_index = row.find('Subject: ')
#                     to_text = row[start_index:end_index]
#                     to_emails = re.findall(r'<(.*?)>', to_text)
#                     sent_to.append(to_emails) # save output
#                     sent_to_how_many.append(len(to_emails)) # save output
#                     row = re.sub(to_text,"",row)

# #                     print(f'To:\t\t{to_emails}')
# #                     print(f'emails sent to: {len(to_emails)} people')

                
                    
#                     start_index = row.find('Sent: ') + len('Sent: ')
#                     end_index = row.find('To: ')
#                     date_text = row[start_index:end_index]
                
# #                     print(f'subject:\t{this_subject}')


#                     start_index = row.find('Subject: ') + len('Subject: ')
#                     end_index = row.find('      ')
#                     this_subject = row[start_index:end_index]
#                     subject.append(this_subject) # save output


#                     row = re.sub('Sent: ',"",row)
#                     row = re.sub('To: ',"",row)
#                     row = re.sub('Subject: ',"",row)
#                     row = re.sub(this_subject,"",row)
                    
# #                     # save the text details
# #                     from_name.append(this_person) # save output

# #                     if not(this_email):
# #                         this_person = before_sent
# #                         from_address.append(this_person) # save output
# #                     else:
# #                         from_address.append(this_email) # save output
# #                         this_person = re.sub(r'<(.*?)>',"",before_sent)  
                        
# #                     internal_date.append(date_text) # save output
# #                     sent_to.append(to_emails) # save output
# #                     sent_to_how_many.append(len(to_emails)) # save output
# #                     subject.append(this_subject) # save output

#                     # conditionals
#                     sort_code_bool, sort_code_num = find_sort_code_in_email(row)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(row))
#                     mentions_change.append(find_update_words_in_email(row))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(row))
        
#                     has_error.append(False) # ran without error
                
#                 elif i == 0:
                    
#                     # first email in thread, save as if saving from the main data
#                     subject.append(line.subject)
#                     internal_date.append(line.internal_date)
#                     from_address.append(line.from_address)
#                     from_name.append(line.from_address) # just save sender email instead here
#                     sent_to.append(line.to)
#                     sent_to_how_many.append(line.to.count(',')+1)

#                     sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(line.body))
#                     mentions_change.append(find_update_words_in_email(line.body))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(row))
                    
#                     has_error.append(False) # ran without error

                    
#         # email threw an error
#         except re.error as e:
            
#             error_count += 1
#             has_error.append(True) # ran with error
            
#             # need to count if all lists are same length - error could have happened somewhere
#             arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#                      bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#                      email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#                      mentions_bank,mentions_change,mentions_account_specifics,has_error]
            
#             # add None's to emails with an error
#             [iter_count_list,email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#              bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#              email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#              mentions_bank,mentions_change,mentions_account_specifics,has_error] = check_array_lengths_add_Nones(arrays)
            
            
#             error_mail_id.append(line.id)
#             error_mail_body.append(line.body) 
#             error_type.append(e)
            
#             # print out the error emails only
#             print("#" * 70, "\n\n")
#             print(f'Regex error occurred: {e}\n\n')               
#             print(f'email id: {line.id}, iter number: {iter_count}. So far there have been {error_count} errors.\n\n')
#             print(f'subject line:\t{line.subject}')
#             print(f'received from:\t{line.from_address}')
#             print(f'sent to:\t{line.to}')
#             #print(f'emails sent to: {line.to.count(',')+1} people')
#             print(f'date sent:\t{line.internal_date}')
#             print(f'slug:\t\t{line.category_slug}')
#             print("")
#             print(line.body)
#             print("\n")

# print("-" * 50, "\n")
# print(f'there were {error_count} errors in total.\n')
# print("-" * 50)    

In [ ]:
arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
                     bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
                     email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
                     mentions_bank,mentions_change,mentions_account_specifics,has_error]
            
check_array_lengths(arrays)

In [ ]:
dict_data = {'iter_count_list':iter_count_list,'email_id':email_id,'is_part_of_thread':is_part_of_thread,'num_in_thread':num_in_thread,'thread_id':thread_id,
             'has_error':has_error,
             'internal_date':internal_date,'category_id':category_id,'category_slug':category_slug,'subject':subject,
            'has_attachment':has_attachment,'has_sort_code':has_sort_code,'sort_code':sort_code,
             'mentions_bank':mentions_bank,'mentions_change':mentions_change,'mentions_account_specifics':mentions_account_specifics,
             'from_address':from_address,'from_name':from_name,'sent_to':sent_to,'sent_to_how_many':sent_to_how_many,
             'bcc':bcc,'cc':cc,'reply_to_address':reply_to_address,'attachments':attachments,'email_text_body':email_text_body
            }

In [ ]:
# dict_data = {'iter_count_list':iter_count_list,'email_id':email_id,'is_part_of_thread':is_part_of_thread,'num_in_thread':num_in_thread,'thread_id':thread_id,
#              'has_error':has_error,
#              'category_id':category_id,'category_slug':category_slug,'internal_date':internal_date
#             }

# print(len(dict_data))

In [ ]:
check_dict_value_lengths(dict_data)

In [ ]:
new_df = pd.DataFrame.from_dict(dict_data)

In [ ]:

new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']] = new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']].fillna(0).astype(int)

In [ ]:
new_df

In [ ]:
new_df.to_csv('unwrapped_labelled_emails.csv')

In [ ]:
new_df.to_pickle("labelled_emails_split_newfeatures.pkl")

## Some error handling

In [ ]:
new_df.loc[new_df['email_id'] == 207972, 'email_text_body']

In [ ]:
new_df.loc[new_df['email_id'] == 207972, 'subject']

In [ ]:
new_df.loc[new_df['email_id'] == 207972, 'internal_date']

In [ ]:
new_df.loc[new_df['email_id'] == 207972, 'internal_date']

In [ ]:
# labelled_emails["body"].iloc[2]

# This is function 2 to check it

In [ ]:
# grow lists iteratively, then create dataframe at the end
email_id = []
subject = []
internal_date = []
from_address = []
from_name = []
sent_to = []
sent_to_how_many = []
bcc = []
cc = []
reply_to_address = []
attachments = []
category_id = []
category_slug = []
has_attachment = []
email_text_body = []

# new columns for splitting the emails
is_part_of_thread = []
num_in_thread = []
thread_id = []

# sort code related
has_sort_code = []
sort_code = []

# language related
mentions_bank = []
mentions_change = []
mentions_account_specifics = []

# error emails:
error_mail_id = []
error_mail_body = []
# error_type = []
# error_value = []
# error_traceback = []

############################
# splitting emails by this: # Disclaimer: need to check this is the same across languages etc
split_string = "From: "

iter_count = 1
error_count = 0

# change name of df here
for line in labelled_emails[0:100].itertuples():
    
#     print("#" * 70, "\n\n")
#     print(f'email {iter_count} in dataframe, email id: {line.id}')
    iter_count += 1
    
    ################### if single email, we just append as we would
    if split_string not in line.body:
        
        # save thread details
        is_part_of_thread.append(0)
        num_in_thread.append(None)
        thread_id.append(None)
        
        # save data
        email_id.append(line.id)
        attachments.append(line.attachments)
        category_id.append(line.category_id)
        category_slug.append(line.category_slug)
        has_attachment.append(line.has_attachment)
        
        # email specific
        subject.append(line.subject)
        internal_date.append(line.internal_date)
        from_address.append(line.from_address)
        from_name.append(line.from_address) # just save sender email instead here
        sent_to.append(line.to)
        sent_to_how_many.append(line.to.count(',')+1)
        bcc.append(line.bcc)
        cc.append(line.cc)
        reply_to_address.append(line.reply_to)
      
        sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
        has_sort_code.append(sort_code_bool)
        sort_code.append(sort_code_num)

        # language related
        mentions_bank.append(find_bank_mention_in_email(line.body))
        mentions_change.append(find_update_words_in_email(line.body))
        mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

#         print(f'subject line:\t{line.subject}')
#         print(f'received from:\t{line.from_address}')
#         print(f'sent to:\t{line.to}')
#         #print(f'emails sent to: {line.to.count(',')+1} people')
#         print(f'date sent:\t{line.internal_date}')
#         print(f'slug:\t\t{line.category_slug}')
#         print("")
#         print(clean_text_re_links_brackets(line.body))
#         print("\n")
        
        # clean email body
        email_text_body.append(clean_text_re_links_brackets(line.body))
        
    ################### thread of emails
    else:
        split_emails = line.body.split("From: ")
        
        # try / except handling here
        try:
            for i, row in enumerate(split_emails):

                row = row.replace('/',' ')
                
                
                 # save thread details
                is_part_of_thread.append(1)
                num_in_thread.append(int(len(split_emails)-(i+1)))
                thread_id.append(str(line.id)+'_t')

                # save regardless of thread or not
                email_id.append(line.id)
                attachments.append(line.attachments)
                category_id.append(line.category_id)
                category_slug.append(line.category_slug)
                has_attachment.append(line.has_attachment)

                bcc.append(None)
                cc.append(None)
                reply_to_address.append(line.reply_to) # just keep this consistent?


                # for every split email which wasn't the first
                if i > 0:

                    before_sent = row.split("Sent: ")[0]
#                     print(f'before sent: {before_sent}\n')
                    #print(f'From: {this_person}, {this_email}')
                    this_email = re.findall(r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])', before_sent)
#                     print(f'from email:\t{this_email}')

#                     print(f'body: {row}')
                    row = re.sub(before_sent,"",row)
                    #row = row.split("Sent: ")[1]

#                     if not(this_email):
#                         this_person = before_sent
#                         from_address.append(this_person) # save output
#                     else:
#                         from_address.append(this_email) # save output
#                         this_person = re.sub(r'<(.*?)>',"",before_sent)  

#                     print(f'from person:\t{this_person}')
#                     from_name.append(this_person) # save output
                    start_index = row.find('Sent: ') + len('Sent: ')
                    end_index = row.find('To: ')
                    date_text = row[start_index:end_index]
#                     internal_date.append(date_text) # save output

                    row = re.sub(date_text,"",row)
                    #row = row.split('To: ')[1]
#                     print(f'Date:\t\t{date_text}')

                    start_index = row.find('To: ') + len('To: ')
                    end_index = row.find('Subject: ')
                    to_text = row[start_index:end_index]
                    to_emails = re.findall(r'<(.*?)>', to_text)
#                     sent_to.append(to_emails) # save output
                    row = re.sub(to_text,"",row)

#                     print(f'To:\t\t{to_emails}')
#                     print(f'emails sent to: {len(to_emails)} people')
#                     sent_to_how_many.append(len(to_emails)) # save output
                    start_index = row.find('Subject: ') + len('Subject: ')
                    end_index = row.find('      ')
                    this_subject = row[start_index:end_index]
#                     print(f'subject:\t{this_subject}')
#                     subject.append(this_subject) # save output


                    row = re.sub('Sent: ',"",row)
                    row = re.sub('To: ',"",row)
                    row = re.sub('Subject: ',"",row)
                    row = re.sub(this_subject,"",row)
                    
                    # save the text details
                    from_name.append(this_person) # save output

                    if not(this_email):
                        this_person = before_sent
                        from_address.append(this_person) # save output
                    else:
                        from_address.append(this_email) # save output
                        this_person = re.sub(r'<(.*?)>',"",before_sent)  
                        
                    internal_date.append(date_text) # save output
                    sent_to.append(to_emails) # save output
                    sent_to_how_many.append(len(to_emails)) # save output
                    subject.append(this_subject) # save output

                    
#                     # save thread details
#                     is_part_of_thread.append(1)
#                     num_in_thread.append(int(len(split_emails)-(i+1)))
#                     thread_id.append(str(line.id)+'_t')

#                     # save regardless of thread or not
#                     email_id.append(line.id)
#                     attachments.append(line.attachments)
#                     category_id.append(line.category_id)
#                     category_slug.append(line.category_slug)
#                     has_attachment.append(line.has_attachment)

#                     bcc.append(None)
#                     cc.append(None)
#                     reply_to_address.append(line.reply_to) # just keep this consistent?

                    # conditionals
                    sort_code_bool, sort_code_num = find_sort_code_in_email(row)
                    has_sort_code.append(sort_code_bool)
                    sort_code.append(sort_code_num)

                    # language related
                    mentions_bank.append(find_bank_mention_in_email(row))
                    mentions_change.append(find_update_words_in_email(row))
                    mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

#                     print('\n')
#                     print(clean_text_re_links_brackets(row))
#                     print('\n')

                    # clean email body
                    email_text_body.append(clean_text_re_links_brackets(row))
                
                elif i == 0:
#                     print(f'subject line:\t{line.subject}')
#                     print(f'received from:\t{line.from_address}')
#                     print(f'sent to:\t{line.to}')
#                     #print(f'emails sent to: {line.to.count(',')+1} people')
#                     print(f'date sent:\t{line.internal_date}')
#                     print(f'slug:\t\t{line.category_slug}')

#                      # save thread details
#                     is_part_of_thread.append(1)
#                     num_in_thread.append(int(len(split_emails)-(i+1)))
#                     thread_id.append(str(line.id)+'_t')

#                     # save regardless of thread or not
#                     email_id.append(line.id)
#                     attachments.append(line.attachments)
#                     category_id.append(line.category_id)
#                     category_slug.append(line.category_slug)
#                     has_attachment.append(line.has_attachment)

#                     bcc.append(None)
#                     cc.append(None)
#                     reply_to_address.append(line.reply_to) # just keep this consistent?


                    subject.append(line.subject)
                    internal_date.append(line.internal_date)
                    from_address.append(line.from_address)
                    from_name.append(line.from_address) # just save sender email instead here
                    sent_to.append(line.to)
                    sent_to_how_many.append(line.to.count(',')+1)

                    sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
                    has_sort_code.append(sort_code_bool)
                    sort_code.append(sort_code_num)

                    # language related
                    mentions_bank.append(find_bank_mention_in_email(line.body))
                    mentions_change.append(find_update_words_in_email(line.body))
                    mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

                    # clean email body
                    email_text_body.append(clean_text_re_links_brackets(row))

#                     print("")
#                     print(clean_text_re_links_brackets(row))
#                     print('\n')
                    
        # email threw an error
        except Exception as e:
            error_count += 1
            
#             # need to count if all lists are same length
#             arrays = [email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#                      bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#                      email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#                      mentions_bank,mentions_change,mentions_account_specifics]
#             it = 
#             if not all(len(l) == the_len for l in it):
#              raise ValueError('not all lists have same length!')
            
            
            print("#" * 70, "\n\n")
#             print(f'email {iter_count} in dataframe, email id: {line.id}')
            print(f'\n\n\n\nemail with id: {line.id}, iter number: {iter_count}, threw an error. So far there have been {error_count} errors.\n\n\n\n')
            error_mail_id.append(line.id)
            error_mail_body.append(line.body) 
            
            
            
            #logger.error('Error: ', str(e))
            
#             error_type.append(exc_type)
#             error_value.append(exc_value)
#             error_traceback.append(exc_traceback)
            
            print(f'subject line:\t{line.subject}')
            print(f'received from:\t{line.from_address}')
            print(f'sent to:\t{line.to}')
            #print(f'emails sent to: {line.to.count(',')+1} people')
            print(f'date sent:\t{line.internal_date}')
            print(f'slug:\t\t{line.category_slug}')
            print("")
            print(line.body)
            print("\n")
            
#             # save data
#         email_id.append(line.id)
#         attachments.append(line.attachments)
#         category_id.append(line.category_id)
#         category_slug.append(line.category_slug)
#         has_attachment.append(line.has_attachment)
        
#         # email specific
#         subject.append(line.subject)
#         internal_date.append(line.internal_date)
#         from_address.append(line.from_address)
#         from_name.append(line.from_address) # just save sender email instead here
#         sent_to.append(line.to)
#         sent_to_how_many.append(line.to.count(',')+1)
#         bcc.append(line.bcc)
#         cc.append(line.cc)
#         reply_to_address.append(line.reply_to)

print(f'there were {error_count} errors in total.')

# without try/except error handling

In [ ]:
# grow lists iteratively, then create dataframe at the end
email_id = []
subject = []
internal_date = []
from_address = []
from_name = []
sent_to = []
sent_to_how_many = []
bcc = []
cc = []
reply_to_address = []
attachments = []
category_id = []
category_slug = []
has_attachment = []
email_text_body = []

# new columns for splitting the emails
is_part_of_thread = []
num_in_thread = []
thread_id = []

# sort code related
has_sort_code = []
sort_code = []

# language related
mentions_bank = []
mentions_change = []
mentions_account_specifics = []

############################
# splitting emails by this: # Disclaimer: need to check this is the same across languages etc
split_string = "From: "

iter_count = 1

# change name of df here
for line in labelled_emails[0:100].itertuples():
    
    print("#" * 70, "\n\n")
    print(f'email {iter_count} in dataframe, email id: {line.id}')
    iter_count += 1
    
    ################### if single email, we just append as we would
    if split_string not in line.body:
        
        # save thread details
        is_part_of_thread.append(0)
        num_in_thread.append(None)
        thread_id.append(None)
        
        # save data
        email_id.append(line.id)
        attachments.append(line.attachments)
        category_id.append(line.category_id)
        category_slug.append(line.category_slug)
        has_attachment.append(line.has_attachment)
        
        # email specific
        subject.append(line.subject)
        internal_date.append(line.internal_date)
        from_address.append(line.from_address)
        from_name.append(line.from_address) # just save sender email instead here
        sent_to.append(line.to)
        sent_to_how_many.append(line.to.count(',')+1)
        bcc.append(line.bcc)
        cc.append(line.cc)
        reply_to_address.append(line.reply_to)
      
        sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
        has_sort_code.append(sort_code_bool)
        sort_code.append(sort_code_num)

        # language related
        mentions_bank.append(find_bank_mention_in_email(line.body))
        mentions_change.append(find_update_words_in_email(line.body))
        mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

        print(f'subject line:\t{line.subject}')
        print(f'received from:\t{line.from_address}')
        print(f'sent to:\t{line.to}')
        #print(f'emails sent to: {line.to.count(',')+1} people')
        print(f'date sent:\t{line.internal_date}')
        print(f'slug:\t\t{line.category_slug}')
        print("")
        print(clean_text_re_links_brackets(line.body))
        print("\n")
        
        # clean email body
        email_text_body.append(clean_text_re_links_brackets(line.body))
        
    ################### thread of emails
    else:
        split_emails = line.body.split("From: ")
        
        for i, row in enumerate(split_emails):
            
            row = row.replace('/',' ')
            
             # save thread details
            is_part_of_thread.append(1)
            num_in_thread.append(int(len(split_emails)-(i+1)))
            thread_id.append(str(line.id)+'_t')
            
            # save regardless of thread or not
            email_id.append(line.id)
            attachments.append(line.attachments)
            category_id.append(line.category_id)
            category_slug.append(line.category_slug)
            has_attachment.append(line.has_attachment)
            
            bcc.append(None)
            cc.append(None)
            reply_to_address.append(line.reply_to) # just keep this consistent?
            
            # for every split email which wasn't the first
            if i > 0:
                                  
                before_sent = row.split("Sent: ")[0]
                print(f'before sent: {before_sent}\n')
                #print(f'From: {this_person}, {this_email}')
                this_email = re.findall(r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])', before_sent)
                print(f'from email:\t{this_email}')
                
                print(f'body: {row}')
                row = re.sub(before_sent,"",row)
                #row = row.split("Sent: ")[1]
                
                if not(this_email):
                    this_person = before_sent
                    from_address.append(this_person) # save output
                else:
                    from_address.append(this_email) # save output
                    this_person = re.sub(r'<(.*?)>',"",before_sent)  
                
                print(f'from person:\t{this_person}')
                from_name.append(this_person) # save output
                start_index = row.find('Sent: ') + len('Sent: ')
                end_index = row.find('To: ')
                date_text = row[start_index:end_index]
                internal_date.append(date_text) # save output
                
                row = re.sub(date_text,"",row)
                #row = row.split('To: ')[1]
                print(f'Date:\t\t{date_text}')

                start_index = row.find('To: ') + len('To: ')
                end_index = row.find('Subject: ')
                to_text = row[start_index:end_index]
                to_emails = re.findall(r'<(.*?)>', to_text)
                sent_to.append(to_emails) # save output
                row = re.sub(to_text,"",row)
                
                print(f'To:\t\t{to_emails}')
                print(f'emails sent to: {len(to_emails)} people')
                sent_to_how_many.append(len(to_emails)) # save output
                start_index = row.find('Subject: ') + len('Subject: ')
                end_index = row.find('      ')
                this_subject = row[start_index:end_index]
                print(f'subject:\t{this_subject}')
                subject.append(this_subject) # save output
                
                
                row = re.sub('Sent: ',"",row)
                row = re.sub('To: ',"",row)
                row = re.sub('Subject: ',"",row)
                row = re.sub(this_subject,"",row)
                
                # conditionals
                sort_code_bool, sort_code_num = find_sort_code_in_email(row)
                has_sort_code.append(sort_code_bool)
                sort_code.append(sort_code_num)

                # language related
                mentions_bank.append(find_bank_mention_in_email(row))
                mentions_change.append(find_update_words_in_email(row))
                mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

                print('\n')
                print(clean_text_re_links_brackets(row))
                print('\n')
                
                # clean email body
                email_text_body.append(clean_text_re_links_brackets(row))
                
            elif i == 0:
                print(f'subject line:\t{line.subject}')
                print(f'received from:\t{line.from_address}')
                print(f'sent to:\t{line.to}')
                #print(f'emails sent to: {line.to.count(',')+1} people')
                print(f'date sent:\t{line.internal_date}')
                print(f'slug:\t\t{line.category_slug}')
                
                
                subject.append(line.subject)
                internal_date.append(line.internal_date)
                from_address.append(line.from_address)
                from_name.append(line.from_address) # just save sender email instead here
                sent_to.append(line.to)
                sent_to_how_many.append(line.to.count(',')+1)
                
                sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
                has_sort_code.append(sort_code_bool)
                sort_code.append(sort_code_num)

                # language related
                mentions_bank.append(find_bank_mention_in_email(line.body))
                mentions_change.append(find_update_words_in_email(line.body))
                mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
                
                # clean email body
                email_text_body.append(clean_text_re_links_brackets(row))

                print("")
                print(clean_text_re_links_brackets(row))
                print('\n')